In [1]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt

import aot


In [2]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)
total_video_number = core_settings["various"]["total_video_number"]
settings_root_path = base_dir / core_settings["paths"]["settings_path"]
main_settings_path = settings_root_path / "main"

video_names = []
for i in range(1, 1 + total_video_number):
    # 0 padding for video name
    video_name = str(i).zfill(4) + ".mp4"
    video_names.append(video_name)
print("videos number: ", len(video_names))

origin_videos_number = len(video_names)
subject_number = core_settings["various"]["subject_number"]
print("subject number: ", subject_number)
session_number = core_settings["various"]["session_number"]
print("session number: ", session_number)
run_number = core_settings["various"]["run_number"]
print("run number: ", run_number)
# only for main run
blank_number_per_run = core_settings["various"]["blank_number_per_run"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
video_number_per_run_main = int((unique_video_number_per_session * 4) / run_number)
# only 1 subject and 1 session for grading
video_number_per_run_grading = int(origin_videos_number / run_number)

videos number:  2179


In [3]:
def read_settings_file(subject, ses, run):
    settings_file = main_settings_path / f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(ses).zfill(2)}_run_{str(run).zfill(2)}.yml"
    settings = yaml.load(open(settings_file), Loader=yaml.FullLoader)
    return settings

In [4]:
def read_settings_files_session(subject, ses):#return a list of settings files for a given session
    settings_files = [read_settings_file(subject, ses, run) for run in range(1,run_number+1)]
    sessions_movies = [settings['stimuli']['movie_files'] for settings in settings_files]
    return sessions_movies

In [5]:
def check_video_replay_of_session(session_movies):
    #check if the same video is replayed in a session
    #append all the lists of movies in a session
    all_movies = []
    for run_movies in session_movies:
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    #check if there is a replay
    stack = []
    for movie in all_movies:
        if movie in stack:
            stack.remove(movie)
        else:
            stack.append(movie)
    return len(stack) == 0

def check_video_replay_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_replay = check_video_replay_of_session(session_movies)
            print(f"subject {subject}, session {ses} replay: {is_replay}")

check_video_replay_all_session()

subject 1, session 1 replay: True
subject 1, session 2 replay: True
subject 1, session 3 replay: True
subject 1, session 4 replay: True
subject 1, session 5 replay: True
subject 1, session 6 replay: True
subject 1, session 7 replay: True
subject 1, session 8 replay: True
subject 1, session 9 replay: True


subject 1, session 10 replay: True
subject 1, session 11 replay: True
subject 1, session 12 replay: True
subject 2, session 1 replay: True
subject 2, session 2 replay: True
subject 2, session 3 replay: True
subject 2, session 4 replay: True
subject 2, session 5 replay: True
subject 2, session 6 replay: True
subject 2, session 7 replay: True
subject 2, session 8 replay: True
subject 2, session 9 replay: True
subject 2, session 10 replay: True
subject 2, session 11 replay: True
subject 2, session 12 replay: True
subject 3, session 1 replay: True
subject 3, session 2 replay: True
subject 3, session 3 replay: True
subject 3, session 4 replay: True
subject 3, session 5 replay: True
subject 3, session 6 replay: True
subject 3, session 7 replay: True
subject 3, session 8 replay: True
subject 3, session 9 replay: True
subject 3, session 10 replay: True
subject 3, session 11 replay: True
subject 3, session 12 replay: True
subject 4, session 1 replay: True
subject 4, session 2 replay: True
subje

In [6]:
def check_video_direction_duality_of_session(session_movies):
    #check whether the videos in a session are duality pairs(R_S_movie,S_movie)
    #append all the lists of movies in a session
    def check_duality(video1,video2):
        '''
        if video1.startswith('R_') and video2.startswith('S_'):
            return video1[4:] == video2[2:]
        elif video1.startswith('S_') and video2.startswith('R_'):
            return video1[2:] == video2[4:]
        else:
            return False
        '''
        if video1.endswith('_fw.mp4') and video2.endswith('_rv.mp4'):
            return video1[:-7] == video2[:-7]
        elif video1.endswith('_rv.mp4') and video2.endswith('_fw.mp4'):
            return video1[:-7] == video2[:-7]
        else:
            return False
    def is_duality_in_stack(video, video_stack):#with side effect that remove the duality pair
        for v in video_stack:
            if check_duality(video,v):
                video_stack.remove(v)
                return True
        return False
    all_movies = []
    for run_movies in session_movies: 
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    stack = []
    for movie in all_movies:
        if not is_duality_in_stack(movie, stack):
            stack.append(movie)
    return len(stack) == 0

def check_video_direction_duality_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_duality = check_video_direction_duality_of_session(session_movies)
            print(f"subject {subject}, session {ses} duality: {is_duality}")

check_video_direction_duality_all_session()
    

subject 1, session 1 duality: True
subject 1, session 2 duality: True


subject 1, session 3 duality: True
subject 1, session 4 duality: True
subject 1, session 5 duality: True
subject 1, session 6 duality: True
subject 1, session 7 duality: True
subject 1, session 8 duality: True
subject 1, session 9 duality: True
subject 1, session 10 duality: True
subject 1, session 11 duality: True
subject 1, session 12 duality: True
subject 2, session 1 duality: True
subject 2, session 2 duality: True
subject 2, session 3 duality: True
subject 2, session 4 duality: True
subject 2, session 5 duality: True
subject 2, session 6 duality: True
subject 2, session 7 duality: True
subject 2, session 8 duality: True
subject 2, session 9 duality: True
subject 2, session 10 duality: True
subject 2, session 11 duality: True
subject 2, session 12 duality: True
subject 3, session 1 duality: True
subject 3, session 2 duality: True
subject 3, session 3 duality: True
subject 3, session 4 duality: True
subject 3, session 5 duality: True
subject 3, session 6 duality: True
subject 3, ses

In [7]:
def count_same_video_in_one_run(subject, ses, run):
    #count the number of same video in one run
    settings = read_settings_file(subject, ses, run)
    movies = settings['stimuli']['movie_files']
    movies = [movie for movie in movies if movie != 'blank']
    count = 0
    stack = []
    for movie in movies:
        if movie in stack:
            stack.remove(movie)
            count += 1
        else:
            stack.append(movie)
    return count

def count_same_video_in_all_run():
    count_list = []
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            for run in range(1,run_number+1):
                count = count_same_video_in_one_run(subject, ses, run)
                count_list.append(count)
                print(f"subject {subject}, session {ses} ,run {run} same video count: {count}")
    return count_list

count_same_video_in_all_run()


subject 1, session 1 ,run 1 same video count: 0
subject 1, session 1 ,run 2 same video count: 0
subject 1, session 1 ,run 3 same video count: 1
subject 1, session 1 ,run 4 same video count: 0
subject 1, session 1 ,run 5 same video count: 0
subject 1, session 1 ,run 6 same video count: 0
subject 1, session 1 ,run 7 same video count: 0
subject 1, session 1 ,run 8 same video count: 0
subject 1, session 1 ,run 9 same video count: 0
subject 1, session 1 ,run 10 same video count: 0
subject 1, session 1 ,run 11 same video count: 0
subject 1, session 1 ,run 12 same video count: 0
subject 1, session 1 ,run 13 same video count: 0
subject 1, session 1 ,run 14 same video count: 0
subject 1, session 1 ,run 15 same video count: 0
subject 1, session 1 ,run 16 same video count: 0
subject 1, session 1 ,run 17 same video count: 0
subject 1, session 1 ,run 18 same video count: 0
subject 1, session 1 ,run 19 same video count: 0
subject 1, session 1 ,run 20 same video count: 0
subject 1, session 1 ,run 21 

subject 1, session 5 ,run 32 same video count: 0
subject 1, session 5 ,run 33 same video count: 0
subject 1, session 5 ,run 34 same video count: 0
subject 1, session 5 ,run 35 same video count: 0
subject 1, session 5 ,run 36 same video count: 0
subject 1, session 5 ,run 37 same video count: 0
subject 1, session 5 ,run 38 same video count: 0
subject 1, session 5 ,run 39 same video count: 0
subject 1, session 5 ,run 40 same video count: 0
subject 1, session 5 ,run 41 same video count: 0
subject 1, session 5 ,run 42 same video count: 0
subject 1, session 5 ,run 43 same video count: 1
subject 1, session 5 ,run 44 same video count: 0
subject 1, session 5 ,run 45 same video count: 0
subject 1, session 5 ,run 46 same video count: 0
subject 1, session 5 ,run 47 same video count: 0
subject 1, session 5 ,run 48 same video count: 1
subject 1, session 5 ,run 49 same video count: 0
subject 1, session 5 ,run 50 same video count: 0
subject 1, session 5 ,run 51 same video count: 0
subject 1, session 5

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [8]:
def check_uniqe_video_number_bothdirection(sub, ses):
    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    print(videoset)
    print("unique video number: ", len(videoset))
    return videoset


def check_uniqe_video_number_uniqe_original(sub, ses):
    def desuffix(name):
        return name[:-3] #remove the suffix _fw or _rv

    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    new_video_list = []
    for video in videoset:
        new_video_list.append(desuffix(video))

    videoset = set(new_video_list)
    print(
        "unique video number sub {sub} ses {ses}: ".format(sub=sub, ses=ses),
        len(videoset),
    )

    return videoset
    

In [9]:
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_bothdirection(sub, ses)

{'0744_fw', '1295_rv', '1079_rv', '1259_rv', '0352_fw', '0877_rv', '0305_fw', '0322_fw', '0754_rv', '0309_fw', '0130_rv', '0188_fw', '0679_rv', '0528_rv', '0202_rv', '0748_rv', '2151_rv', '1621_rv', '2045_rv', '0865_rv', '0661_rv', '0685_fw', '1771_rv', '0799_rv', '0935_fw', '1731_rv', '0322_rv', 'blank', '2123_fw', '0064_fw', '1533_fw', '0090_rv', '1719_rv', '0777_rv', '1486_fw', '2004_rv', '0309_rv', '1389_rv', '0305_rv', '0888_rv', '1475_rv', '1042_fw', '1245_fw', '0643_fw', '1663_fw', '0494_fw', '1043_rv', '0846_rv', '2113_rv', '1308_fw', '1696_fw', '1806_rv', '1914_rv', '2085_rv', '1955_fw', '2126_fw', '1238_fw', '1102_fw', '0341_rv', '1261_rv', '1802_fw', '1951_fw', '1384_fw', '1895_fw', '2008_rv', '1254_rv', '1620_fw', '0909_fw', '1093_fw', '2023_fw', '1799_rv', '1148_rv', '2091_rv', '0818_rv', '0806_rv', '0661_fw', '0846_fw', '1385_rv', '0075_rv', '0135_fw', '0679_fw', '0188_rv', '0228_fw', '1379_fw', '0530_fw', '1305_rv', '0028_rv', '0783_fw', '1081_rv', '0581_rv', '1890_fw', 

{'1340_fw', '0600_rv', '1735_rv', '1546_rv', '1597_rv', '1344_rv', '0208_fw', '0305_fw', '0754_rv', '0309_fw', '1930_rv', '1616_fw', '0236_fw', '0512_fw', '1110_fw', '1808_fw', '0685_fw', '0110_fw', '0007_rv', '1340_rv', '1979_fw', '0935_fw', '0204_fw', '0571_rv', 'blank', '1367_rv', '1387_rv', '1076_fw', '0450_rv', '2157_fw', '0721_rv', '0049_rv', '1515_fw', '1905_fw', '0309_rv', '0924_rv', '0199_rv', '0435_rv', '0305_rv', '0435_fw', '0888_rv', '0317_fw', '1673_rv', '1387_fw', '0724_fw', '2121_rv', '1779_rv', '0562_fw', '1043_rv', '1594_rv', '0272_fw', '1571_rv', '0598_fw', '1818_fw', '1828_fw', '1515_rv', '0729_rv', '0358_fw', '1452_fw', '0901_fw', '0323_rv', '2053_fw', '1199_rv', '1614_fw', '0640_rv', '1360_fw', '0367_fw', '0450_fw', '1722_rv', '1260_fw', '1619_fw', '1238_fw', '1424_fw', '1651_rv', '1620_fw', '0105_rv', '1722_fw', '0488_fw', '0367_rv', '0680_rv', '0256_rv', '0630_rv', '0292_rv', '0779_rv', '1557_fw', '1593_rv', '1367_fw', '0989_fw', '1983_rv', '0302_fw', '1159_fw', 

In [10]:
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_uniqe_original(sub, ses)

unique video number sub 1 ses 1:  181
unique video number sub 1 ses 2:  181
unique video number sub 1 ses 3:  181


unique video number sub 1 ses 4:  181
unique video number sub 1 ses 5:  181
unique video number sub 1 ses 6:  181
unique video number sub 1 ses 7:  181
unique video number sub 1 ses 8:  181
unique video number sub 1 ses 9:  181
unique video number sub 1 ses 10:  181
unique video number sub 1 ses 11:  181
unique video number sub 1 ses 12:  181
unique video number sub 2 ses 1:  181
unique video number sub 2 ses 2:  181
unique video number sub 2 ses 3:  181
unique video number sub 2 ses 4:  181
unique video number sub 2 ses 5:  181
unique video number sub 2 ses 6:  181
unique video number sub 2 ses 7:  181
unique video number sub 2 ses 8:  181
unique video number sub 2 ses 9:  181
unique video number sub 2 ses 10:  181
unique video number sub 2 ses 11:  181
unique video number sub 2 ses 12:  181
unique video number sub 3 ses 1:  181
unique video number sub 3 ses 2:  181
unique video number sub 3 ses 3:  181
unique video number sub 3 ses 4:  181
unique video number sub 3 ses 5:  181
unique

In [11]:
def session_unique_video_overlapping_check(sub, ses1, ses2):
    #check the overlapping of unique videos between different sessions
    videoset1 = check_uniqe_video_number_uniqe_original(sub, ses1)
    videoset2 = check_uniqe_video_number_uniqe_original(sub, ses2)
    print("overlapping: ", videoset1 & videoset2)

def all_session_unique_video_overlapping_check():
    for sub in range(1, subject_number + 1):
        for ses1 in range(1, session_number + 1):
            for ses2 in range(ses1 + 1, session_number + 1):
                session_unique_video_overlapping_check(sub, ses1, ses2)

In [12]:
all_session_unique_video_overlapping_check()

unique video number sub 1 ses 1:  181
unique video number sub 1 ses 2:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 3:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 4:  181
overlapping:  {'bl'}


unique video number sub 1 ses 1:  181
unique video number sub 1 ses 5:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 6:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 7:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 8:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 9:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 10:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 11:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 12:  181
overlapping:  {'bl'}
unique video number sub 1 ses 2:  181
unique video number sub 1 ses 3:  181
overlapping:  {'bl'}
unique video number sub 1 ses 2:  181
unique video number sub 1 ses 4:  181
overlapping:  {'bl'}
unique video number sub 1 s